In [1]:
import React, { useState, useEffect, useRef } from 'react';
import { Book, Wand2, Battery, Send, RefreshCw, ShoppingBag, MessageCircle, Shield, Heart, Zap, Brain, Lightbulb, Target, Smile } from 'lucide-react';

// ==========================================
// Google Gemini API 설정
// ==========================================
const apiKey = ""; // 런타임 환경에서 자동으로 주입됩니다.

// ==========================================
// 데이터베이스: Class 1 시나리오 (The Hungry Gatekeeper)
// ==========================================
const scenarioData = {
  title: "Class 1: The Hungry Gatekeeper",
  intro: "마법 학교로 가는 문을 거대한 곰이 막고 있습니다! 배가 고파서 난동을 부리는 것 같아요.",
  
  // Part A: Combat Mode (Syntax Focus - S+V+O)
  combatTasks: [
    { 
      id: 'c1', type: 'worked', 
      situation: "곰이 공격하려 합니다! 방어 주문을 따라하세요.", 
      target: "I make a shield.", 
      guide: "따라 쓰세요: I make a shield." 
    },
    { 
      id: 'c2', type: 'completion', 
      situation: "곰이 돌을 던집니다! 받아내야 해요.", 
      target: "I catch the stone.", 
      answerKeyword: "catch",
      guide: "빈칸 채우기: I _____ the stone. (잡다)" 
    },
    { 
      id: 'c3', type: 'creation', 
      situation: "곰이 달려옵니다! 곰을 멈춰세우세요!", 
      target: "I stop the bear.", 
      guide: "곰을 멈추는(그만) 문장을 스스로 만드세요!" 
    }
  ],

  // Part B: Social Mode (Expression Focus - Wants & Needs)
  socialTasks: [
    { 
      id: 's1', type: 'scaffolded', 
      situation: "곰이 진정되었습니다. 상태를 물어보세요.", 
      target: "Are you hungry?", 
      hint: "Hint: Are you ...?",
      guide: "배가 고픈지 물어보세요." 
    },
    { 
      id: 's2', type: 'scaffolded', 
      situation: "곰이 사과 그림을 가리킵니다. 원하는지 물어보세요.", 
      target: "Do you want an apple?", 
      hint: "Hint: Do you want ...?",
      guide: "사과를 원하는지 물어보세요." 
    },
    { 
      id: 's3', type: 'real_world', 
      situation: "[상점] 곰에게 줄 사과를 사야 합니다.", 
      target: "I want an apple, please.", 
      guide: "상점 주인에게 사과를 달라고 말하세요. (want 사용)" 
    },
    { 
      id: 's4', type: 'real_world', 
      situation: "곰에게 사과를 건네주세요.", 
      target: "Eat this apple.", 
      guide: "곰에게 사과를 먹으라고 하거나 주세요." 
    }
  ]
};

// ==========================================
// AI Logic: The AI Scaffolding Cycle
// ==========================================
async function getWizFeedback(userText, mode, currentTask) {
  try {
    const systemPrompt = `
      당신은 AI 튜터 '위즈'입니다. 교육학적 스캐폴딩 이론에 기반하여 학생을 지도합니다.
      
      [현재 상황]
      - Mode: ${mode}
      - Task: ${currentTask.situation}
      - Target: ${currentTask.target}

      [스캐폴딩 사이클 진단 및 선택]
      학습자의 입력(Performance)을 분석하여 다음 4가지 중 하나의 비계(Scaffolding)를 선택(Selection)하여 지원(Support)하세요.

      1. **Conceptual (개념적)**: 지식의 부재(예: 어순 S+V+O 모름)가 원인일 때.
         - 반응 예: "주어(S) 뒤에는 반드시 동사(V)가 와야 해."
      2. **Strategic (전략적)**: 방법의 부재(예: 단어는 아는데 순서가 틀림)가 원인일 때.
         - 반응 예: "단어의 순서를 바꿔볼까? 누가(Who) 행동(Do) 무엇(What) 순서야."
      3. **Metacognitive (메타인지적)**: 반복적인 실수나 미세한 오류일 때 성찰 유도.
         - 반응 예: "아까 맞았던 문장이랑 지금 문장이랑 뭐가 다를까?"
      4. **Motivational (동기적)**: 거의 맞았거나, 좌절할 것 같을 때.
         - 반응 예: "거의 다 왔어! 동사 하나만 고치면 완벽해!"

      [Output JSON Format]
      {
        "isCorrect": boolean,
        "scaffoldingType": "Conceptual" | "Strategic" | "Metacognitive" | "Motivational" | "Success",
        "message": "위즈의 대사 (이모지 포함)",
        "action": "NONE" | "ATTACK" | "DEFEND" | "BUY_ITEM"
      }
    `;

    const response = await fetch(
      `https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-preview-09-2025:generateContent?key=${apiKey}`,
      {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({
          contents: [{ parts: [{ text: `User Input: "${userText}"` }] }],
          systemInstruction: { parts: [{ text: systemPrompt }] },
          generationConfig: { responseMimeType: "application/json" }
        }),
      }
    );

    if (!response.ok) throw new Error("API Error");
    const data = await response.json();
    return JSON.parse(data.candidates[0].content.parts[0].text);

  } catch (error) {
    console.error(error);
    return { isCorrect: false, scaffoldingType: "Motivational", message: "마법 회로 연결 불안정! 다시 시도해줘.", action: "NONE" };
  }
}

// ==========================================
// Main Component
// ==========================================
export default function Class1_Tutorial() {
  // Game State
  const [gameState, setGameState] = useState('INTRO'); 
  const [taskIndex, setTaskIndex] = useState(0);
  const [messages, setMessages] = useState([]);
  const [userInput, setUserInput] = useState('');
  const [loading, setLoading] = useState(false);
  
  // UI Toggles
  const [showGrimoire, setShowGrimoire] = useState(false);
  const [showManaStation, setShowManaStation] = useState(false);
  const [mana, setMana] = useState(50);

  // RPG Elements
  const [monsterHp, setMonsterHp] = useState(100);
  const [monsterState, setMonsterState] = useState('ANGRY'); 
  const [inventory, setInventory] = useState([]);
  const [background, setBackground] = useState('FOREST');

  // Helpers
  const currentModeTasks = gameState === 'COMBAT' ? scenarioData.combatTasks : scenarioData.socialTasks;
  const currentTask = currentModeTasks[taskIndex] || {};
  const messagesEndRef = useRef(null);

  // Init
  useEffect(() => {
    if (gameState === 'INTRO') {
      setMessages([{ sender: 'wiz', text: `안녕! 위즈야. ${scenarioData.intro}` }]);
      setTimeout(() => setGameState('COMBAT'), 2000);
    }
  }, [gameState]);

  useEffect(() => {
    messagesEndRef.current?.scrollIntoView({ behavior: "smooth" });
  }, [messages]);

  // Core Logic
  const handleSendMessage = async () => {
    if (!userInput.trim()) return;

    const newMessages = [...messages, { sender: 'user', text: userInput }];
    setMessages(newMessages);
    setLoading(true);
    setUserInput('');

    const result = await getWizFeedback(userInput, gameState, currentTask);
    setLoading(false);

    setMessages(prev => [...prev, { 
      sender: 'wiz', 
      text: result.message,
      scaffoldingType: result.scaffoldingType 
    }]);

    if (result.isCorrect) {
      if (gameState === 'COMBAT') setMonsterHp(prev => Math.max(0, prev - 34));
      if (result.action === 'BUY_ITEM') {
        setInventory(['Apple']);
        setMessages(prev => [...prev, { sender: 'system', text: "🍎 사과를 획득했습니다!" }]);
      }

      if (taskIndex < currentModeTasks.length - 1) {
        if (gameState === 'SOCIAL' && taskIndex === 1) {
             setBackground('SHOP');
             setMessages(prev => [...prev, { sender: 'system', text: "🏠 상점으로 이동합니다." }]);
        }
        if (gameState === 'SOCIAL' && taskIndex === 2) {
             setBackground('FOREST');
             setMessages(prev => [...prev, { sender: 'system', text: "🌲 다시 곰에게 돌아갑니다." }]);
        }
        setTimeout(() => setTaskIndex(prev => prev + 1), 1000);
      } else {
        if (gameState === 'COMBAT') handleCombatEnd();
        else if (gameState === 'SOCIAL') handleGameEnd();
      }
    }
  };

  const handleCombatEnd = () => {
    setMessages(prev => [...prev, { sender: 'system', text: "✨ 곰이 공격을 멈췄습니다!" }]);
    setMonsterHp(0);
    setMonsterState('SAD');
    setGameState('SOCIAL');
    setTaskIndex(0);
  };

  const handleGameEnd = () => {
    setMonsterState('HAPPY');
    setMessages(prev => [...prev, { sender: 'system', text: "🎉 미션 성공! 문이 열렸습니다!" }]);
    setGameState('END');
  };

  // Sub-Components
  const GrimoireModal = () => (
    <div className="fixed inset-0 bg-black/70 flex items-center justify-center z-50 p-4">
      <div className="bg-amber-100 rounded-lg max-w-lg w-full p-6 border-4 border-amber-800 shadow-2xl relative">
        <button onClick={() => setShowGrimoire(false)} className="absolute top-2 right-2 text-amber-900"><X size={24} /></button>
        <h2 className="text-2xl font-bold text-amber-900 mb-4 flex items-center gap-2"><Book /> 마법 도감 (Grimoire)</h2>
        <div className="space-y-4 text-amber-900">
          <div className="bg-white/60 p-4 rounded border border-amber-300">
            <h3 className="font-bold text-lg mb-2">⚡ 전투 마법 공식 (Syntax)</h3>
            <p className="bg-amber-200 p-2 rounded text-center font-bold mb-2">S (주인공) + V (행동) + O (대상)</p>
            <ul className="list-disc pl-5 text-sm">
              <li><strong>I</strong> (나) + <strong>hit</strong> (때리다) + <strong>the slime</strong> (슬라임)</li>
              <li><strong>I</strong> (나) + <strong>make</strong> (만들다) + <strong>a shield</strong> (방패)</li>
            </ul>
          </div>
          <div className="bg-white/60 p-4 rounded border border-amber-300">
             <h3 className="font-bold text-lg mb-2">🗣️ 소통의 기술 (Expression)</h3>
             <p className="text-sm">원하는 게 있을 땐: <strong>"I want ~"</strong></p>
             <p className="text-sm">질문할 땐: <strong>"Do you want ~?"</strong></p>
          </div>
        </div>
      </div>
    </div>
  );

  const ManaStationModal = () => {
    const [quizIdx, setQuizIdx] = useState(0);
    const quizzes = [{q:"때리다",a:"hit"}, {q:"원하다",a:"want"}, {q:"사과",a:"apple"}, {q:"방패",a:"shield"}];
    const checkAns = (ans) => {
        if(ans === quizzes[quizIdx].a) {
            setMana(m => m + 10);
            if(quizIdx < quizzes.length-1) setQuizIdx(i => i+1);
            else { alert("마나 충전 완료!"); setShowManaStation(false); }
        } else alert("다시 시도해보세요!");
    };
    return (
      <div className="fixed inset-0 bg-black/70 flex items-center justify-center z-50 p-4">
        <div className="bg-indigo-900 text-white rounded-lg max-w-sm w-full p-6 border-4 border-indigo-500 shadow-2xl relative">
          <button onClick={() => setShowManaStation(false)} className="absolute top-2 right-2"><X size={24}/></button>
          <h2 className="text-xl font-bold mb-4 flex items-center gap-2"><Battery className="text-yellow-400"/> 마나 충전소</h2>
          <div className="text-center py-6 bg-indigo-800 rounded-lg mb-4">
            <div className="text-2xl font-bold mb-2">{quizzes[quizIdx].q}</div>
            <div className="text-sm text-gray-400">영어로 무엇일까요?</div>
          </div>
          <div className="grid grid-cols-2 gap-2">
            {["hit", "want", "apple", "shield", "run", "eat"].sort(()=>Math.random()-0.5).slice(0,4).map(w => (
                <button key={w} onClick={() => checkAns(w)} className="bg-indigo-600 hover:bg-indigo-500 p-3 rounded font-bold">{w}</button>
            ))}
            {/* 정답 보장 로직은 간소화를 위해 생략, 실제로는 정답 포함해서 셔플해야 함 */}
            <button onClick={() => checkAns(quizzes[quizIdx].a)} className="bg-indigo-600 hover:bg-indigo-500 p-3 rounded font-bold">{quizzes[quizIdx].a}</button>
          </div>
        </div>
      </div>
    );
  };

  const renderScaffoldingBadge = (type) => {
    switch(type) {
        case 'Conceptual': return <span className="flex items-center gap-1 text-[10px] bg-blue-100 text-blue-800 px-1.5 py-0.5 rounded border border-blue-200"><Brain size={10}/> Conceptual Hint</span>;
        case 'Strategic': return <span className="flex items-center gap-1 text-[10px] bg-green-100 text-green-800 px-1.5 py-0.5 rounded border border-green-200"><Target size={10}/> Strategic Hint</span>;
        case 'Metacognitive': return <span className="flex items-center gap-1 text-[10px] bg-purple-100 text-purple-800 px-1.5 py-0.5 rounded border border-purple-200"><Lightbulb size={10}/> Reflection</span>;
        case 'Motivational': return <span className="flex items-center gap-1 text-[10px] bg-yellow-100 text-yellow-800 px-1.5 py-0.5 rounded border border-yellow-200"><Smile size={10}/> Cheer Up</span>;
        default: return null;
    }
  };

  return (
    <div className="flex flex-col h-screen bg-slate-900 text-slate-100 font-sans overflow-hidden">
      {/* Header */}
      <header className="bg-slate-800 p-3 shadow-lg flex justify-between items-center border-b border-slate-700 z-20">
        <div className="flex items-center gap-2">
          <div className="bg-purple-600 p-2 rounded-full"><Wand2 size={18} /></div>
          <div>
            <h1 className="text-base font-bold text-purple-200">Wiz Academy</h1>
            <div className="text-xs text-slate-400">Mode: <span className="text-yellow-400 font-bold">{gameState}</span></div>
          </div>
        </div>
        <div className="flex gap-3">
            <button onClick={() => setShowManaStation(true)} className="flex items-center gap-1 bg-indigo-900/50 px-3 py-1 rounded-full border border-indigo-500 text-sm hover:bg-indigo-800 transition">
                <Battery className="w-4 h-4 text-yellow-400" /> <span className="text-yellow-200">{mana}</span>
            </button>
            <button onClick={() => setShowGrimoire(true)} className="flex items-center gap-1 bg-amber-900/50 px-3 py-1 rounded-full border border-amber-600 text-sm hover:bg-amber-800 transition">
                <Book className="w-4 h-4 text-amber-400" /> <span className="text-amber-200">Grimoire</span>
            </button>
        </div>
      </header>

      {/* Visual Stage */}
      <div className={`flex-1 relative transition-all duration-1000 flex flex-col items-center justify-center p-4
        ${background === 'FOREST' ? "bg-[url('https://images.unsplash.com/photo-1448375240586-dfd8f3793371?q=80&w=2670&auto=format&fit=crop')]" : "bg-[url('https://images.unsplash.com/photo-1550989460-0adf9ea622e2?q=80&w=2574&auto=format&fit=crop')]"} 
        bg-cover bg-center`}>
        <div className="absolute inset-0 bg-slate-900/70 backdrop-blur-[2px]"></div>

        {/* Character */}
        <div className="z-10 mb-6 transform scale-125 transition-all">
             {background === 'SHOP' ? (
                 <div className="flex flex-col items-center"><div className="bg-amber-800 p-4 rounded-full border-4 border-amber-600"><ShoppingBag size={64} className="text-amber-200"/></div></div>
             ) : (
                 <div className={`flex flex-col items-center ${monsterState === 'ANGRY' ? 'animate-bounce' : ''}`}>
                    <div className={`bg-slate-800 p-4 rounded-full border-4 ${monsterState === 'ANGRY' ? 'border-red-600' : 'border-blue-400'}`}>
                        {monsterState === 'ANGRY' ? <Shield size={64} className="text-red-600"/> : <Heart size={64} className="text-green-400"/>}
                    </div>
                    {gameState === 'COMBAT' && (
                        <div className="mt-2 w-24 h-2 bg-slate-700 rounded-full overflow-hidden"><div className="h-full bg-red-500 transition-all" style={{width:`${monsterHp}%`}}></div></div>
                    )}
                 </div>
             )}
        </div>

        {/* Chat Area */}
        <div className="z-10 w-full max-w-lg h-64 overflow-y-auto space-y-3 px-2 custom-scrollbar mask-image-gradient">
          {messages.map((msg, idx) => (
            <div key={idx} className={`flex ${msg.sender === 'user' ? 'justify-end' : 'justify-start'}`}>
              <div className={`max-w-[85%] rounded-xl p-3 text-sm shadow-lg ${
                msg.sender === 'user' ? 'bg-purple-600 text-white' : 
                msg.sender === 'system' ? 'bg-black/60 text-yellow-300 text-center w-full border border-yellow-500/30' : 
                'bg-slate-800/90 text-slate-200 border border-slate-600'
              }`}>
                {msg.sender === 'wiz' && (
                    <div className="flex justify-between items-center mb-1">
                        <div className="flex items-center gap-1 text-xs text-purple-300 font-bold"><Wand2 size={10}/> Wiz</div>
                        {msg.scaffoldingType && renderScaffoldingBadge(msg.scaffoldingType)}
                    </div>
                )}
                {msg.text}
              </div>
            </div>
          ))}
          <div ref={messagesEndRef} />
        </div>
      </div>

      {/* Footer Area: Procedural Info + Input */}
      <div className="bg-slate-800 p-4 border-t border-slate-700 z-30">
        <div className="max-w-xl mx-auto">
            {/* Procedural Info (Task Guide) */}
            {gameState !== 'END' && gameState !== 'INTRO' && (
                <div className={`mb-3 p-3 rounded-lg border-l-4 shadow-md transition-all
                    ${gameState === 'COMBAT' ? 'bg-red-900/30 border-red-500' : 'bg-blue-900/30 border-blue-500'}`}>
                    <div className="flex justify-between items-center mb-1">
                        <span className={`text-xs font-bold uppercase tracking-wider ${gameState === 'COMBAT' ? 'text-red-300' : 'text-blue-300'}`}>
                            {gameState === 'COMBAT' ? '⚔️ Combat Task' : '🗣️ Social Task'}
                        </span>
                        <span className="text-xs text-slate-400 bg-slate-800 px-2 py-0.5 rounded">
                            Step {taskIndex + 1}
                        </span>
                    </div>
                    <div className="text-sm text-slate-200 font-medium">
                        {currentTask.guide}
                    </div>
                </div>
            )}

            <div className="flex gap-2">
                <input
                    type="text"
                    value={userInput}
                    onChange={(e) => setUserInput(e.target.value)}
                    onKeyPress={(e) => e.key === 'Enter' && handleSendMessage()}
                    placeholder={gameState === 'END' ? "미션 완료!" : "주문을 입력하세요..."}
                    disabled={loading || gameState === 'END'}
                    className="flex-1 bg-slate-900 border border-slate-600 text-white px-4 py-3 rounded-lg focus:ring-2 focus:ring-purple-500 outline-none placeholder-slate-500 transition-all"
                />
                <button 
                    onClick={handleSendMessage} 
                    disabled={loading || !userInput.trim() || gameState === 'END'}
                    className="bg-purple-600 hover:bg-purple-500 disabled:bg-slate-700 text-white p-3 rounded-lg transition-colors flex items-center justify-center min-w-[3rem]"
                >
                    {loading ? <RefreshCw className="animate-spin w-5 h-5"/> : <Send className="w-5 h-5" />}
                </button>
            </div>
        </div>
      </div>

      {showGrimoire && <GrimoireModal />}
      {showManaStation && <ManaStationModal />}
    </div>
  );
}

SyntaxError: invalid syntax (1368465611.py, line 1)